In [5]:
# test_extraction.py

from modules.extraction.processing import Preprocessor
from modules.extraction.embedding import EmbeddingExtractor

from modules.retrieval.index import EmbeddingIndex
from modules.retrieval.search import Searcher


In [ ]:
# !pip install facenet-pytorch
# import sys
# !{sys.executable} -m pip install facenet-pytorch

In [2]:

# Initialize preprocessor and embedding extractor
preprocessor = Preprocessor()
extractor = EmbeddingExtractor(model_name='casia-webface')
# Test with a sample image



In [4]:
from PIL import Image

image_path = 'simclr_resources\probe\Abdullah\Abdullah_0002.jpg'  # Replace with a real image path

preprocessed_image = preprocessor.preprocess(image_path)
embedding = extractor.extract(preprocessed_image)

print("Preprocessed Image Shape:", preprocessed_image.shape)
print("Extracted Embedding Shape:", embedding.shape)  # Should be (512,)



Preprocessed Image Shape: torch.Size([3, 160, 160])
Extracted Embedding Shape: torch.Size([512])


In [6]:

# test_retrieval.py

import os

# Paths
gallery_dir = 'storage/gallery'
probe_image_path = 'simclr_resources/probe/Abdullah\Abdullah_0002.jpg'
catalog_dir = 'storage/catalog'

# Initialize components
# preprocessor = Preprocessor()
# extractor = EmbeddingExtractor(model_name='casia-webface')
embedding_index = EmbeddingIndex(dimension=512)
searcher = Searcher(embedding_index)

# Index all gallery images
for person_folder in os.listdir(gallery_dir):
    folder_path = os.path.join(gallery_dir, person_folder)
    if os.path.isdir(folder_path):
        for image_file in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_file)
            preprocessed_image = preprocessor.preprocess(image_path)
            embedding = extractor.extract(preprocessed_image)
            embedding_index.add(embedding, image_path)

# Save the index
embedding_index.save(catalog_dir)


In [7]:

# Load the index and perform a search
embedding_index.load(catalog_dir)
probe_preprocessed = preprocessor.preprocess(probe_image_path)
probe_embedding = extractor.extract(probe_preprocessed)
results = searcher.search(probe_embedding, k=5)

# Display results
print("Nearest neighbors:")
for result in results:
    print(f"Image Path: {result['image_path']}, Distance: {result['distance']}")


Nearest neighbors:
Image Path: storage/gallery\Abdullah\Abdullah_0003.jpg, Distance: 0.4168059825897217
Image Path: storage/gallery\Paul_Bremer\Paul_Bremer_0010.jpg, Distance: 0.45640718936920166
Image Path: storage/gallery\Alejandro_Toledo\Alejandro_Toledo_0010.jpg, Distance: 0.48411980271339417
Image Path: storage/gallery\John_Kerry\John_Kerry_0005.jpg, Distance: 0.48846742510795593
Image Path: storage/gallery\Mike_Weir\Mike_Weir_0006.jpg, Distance: 0.5392659306526184
